**Recommendation System**

## Importing required libraries

In [ ]:
import pandas as pd
import numpy as np

### 1. Data Preprocessing

In [ ]:
df = pd.read_csv("/content/anime.csv")

Loading the data set.

In [ ]:

print("Dataset Shape:", df.shape)
print("Column Names:", df.columns)
df.head()

Dataset Shape: (12294, 7)
Column Names: Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')


anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266

Displaying the  basic information of loaded data set such as:

->Shape of the data set

->And printing the column names of the data set

->Displays the first five rows of the dataset.


In [ ]:
print(df.isnull().sum())

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64


 Checks the missing values in the data set.

In [ ]:
df.dropna(inplace=True)

Drop rows where the  missing values exist.

In [ ]:
# Recheck missing values
print("After dropping missing values:")
print(df.isnull().sum())

After dropping missing values:
anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64


Recheck missing values after the drop rows in the data set.

In [ ]:
# Unique types of broadcast
print("Unique Types:", df['type'].unique())

Unique Types: ['Movie' 'TV' 'OVA' 'Special' 'Music' 'ONA']


Retrieves and prints the unique values in the 'type' column of a DataFrame, helping identify the different categories or kinds of broadcasts present in the data.

In [ ]:
print(df.describe())

           anime_id        rating       members
count  12017.000000  12017.000000  1.201700e+04
mean   13638.001165      6.478264  1.834888e+04
std    11231.076675      1.023857  5.537250e+04
min        1.000000      1.670000  1.200000e+01
25%     3391.000000      5.890000  2.250000e+02
50%     9959.000000      6.570000  1.552000e+03
75%    23729.000000      7.180000  9.588000e+03
max    34519.000000     10.000000  1.013917e+06


A statistical summary of the DataFrame, including key descriptive statistics such as count, mean, standard deviation, minimum, quartiles, and maximum for numerical columns, providing an overview of the data distribution and variability.

In [ ]:
print(df['genre'].sample(5).values)

['Comedy' 'Action, Adventure, Comedy, Drama, Shounen'
 'Historical, Samurai, Seinen' 'Sports' 'Comedy, Ecchi, School, Seinen']


A randomly selects and displays five sample values from the 'genre' column of the DataFrame, allowing for a quick review of the different genres present in the dataset.

### 2: Feature Extraction

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Split genres by comma and strip spaces
df['genre'] = df['genre'].apply(lambda x: [i.strip() for i in x.split(',')])

# Use MultiLabelBinarizer to one-hot encode genres
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre'])

# Convert to DataFrame
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

# Combine with ratings
features = pd.concat([genre_df, df['rating']], axis=1)

# Fill any remaining NaNs
features.fillna(0, inplace=True)


Processes the 'genre' column by splitting comma-separated genres into lists and strips spaces. It then applies MultiLabelBinarizer to one-hot encode these genres, converting them into a binary format. The encoded genre data is combined with the 'rating' column, and any remaining missing values are filled with zeros, preparing a comprehensive feature set for subsequent analysis.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features['rating'] = scaler.fit_transform(features[['rating']])


Applies MinMaxScaler to normalize the 'rating' column within the feature set, scaling its values to a range between 0 and 1 for consistent comparability and improved performance in subsequent analyses.

### 3:Recommendation System:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
similarity_matrix = cosine_similarity(features)


calculates a cosine similarity matrix for the feature set, measuring the pairwise similarity between data points based on their feature vectors.

In [ ]:
# Reset index to align with original DataFrame
df = df.reset_index(drop=True)

# Recommendation function
def recommend_anime(anime_title, top_n=5):
    if anime_title not in df['name'].values:
        return f"Anime '{anime_title}' not found in the dataset."

    # Get the index of the anime
    idx = df[df['name'] == anime_title].index[0]

    # Get similarity scores
    sim_scores = list(enumerate(similarity_matrix[idx]))

    # Sort by similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]

    # Get recommended anime names
    recommended_titles = [df.iloc[i[0]]['name'] for i in sim_scores]

    return recommended_titles


Defines a recommender function for anime, which resets the DataFrame index to match the original data, then, given an anime title, finds its index and calculates similarity scores with other entries using a precomputed similarity matrix. It sorts the scores to identify the top similar anime, excluding the input itself, and returns a list of recommended anime titles based on the highest similarity scores.

In [ ]:
recommendations = recommend_anime("Naruto", top_n=5)
print("Recommended for 'Naruto':")
print(recommendations)

Recommended for 'Naruto':
['Naruto: Shippuuden', 'Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi', 'Boruto: Naruto the Movie', 'Naruto x UT', 'Naruto: Shippuuden Movie 4 - The Lost Tower']


Generates and prints a list of the top five anime recommendations similar to "Naruto" using the previously defined recommendation function, providing users with related anime titles based on similarity measures.

### 4: Evaluation

In [ ]:
test_title = "Naruto"
original_genres = df[df['name'] == test_title]['genre'].values[0]
idx = df[df['name'] == test_title].index[0]

# Temporarily remove one genre
if len(original_genres) > 1:
    df.at[idx, 'genre'] = original_genres[:-1]  # Masking

    # Rebuild features
    genre_encoded = mlb.fit_transform(df['genre'])
    genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
    features = pd.concat([genre_df, df['rating']], axis=1)
    features.fillna(0, inplace=True)
    features['rating'] = scaler.fit_transform(features[['rating']])
    similarity_matrix = cosine_similarity(features)

    # Recommend
    recommendations = recommend_anime(test_title, top_n=5)
    print(f"Recommendations after masking a genre in '{test_title}':")
    print(recommendations)

    #Restore original genre
    df.at[idx, 'genre'] = original_genres



Recommendations after masking a genre in 'Naruto':
['Battle Spirits: Ryuuko no Ken', 'Shijou Saikyou no Deshi Kenichi', 'Boruto: Naruto the Movie', 'Shijou Saikyou no Deshi Kenichi OVA', 'Naruto: Shippuuden']


**Interview Questions:**

Temporarily removes the last genre from "Naruto" to assess how this change affects the feature representation and similarity calculations, rebuilding the feature set and recomputing the similarity matrix. It then generates and prints a list of top five anime recommendations based on this modified genre, and finally restores the original genre data.

1.Can you explain the difference between user-based and item-based collaborative filtering?
   
 User-based Collaborative Filtering:

 * This approach recommends items to a user based on the preferences of other users with similar tastes.

 * It assumes: "If User A is similar to User B, then User A is likely to enjoy items that User B likes."

 * Example: If you and another user both rated many action anime highly, the system might recommend to you an anime that the other user rated well but you haven't seen yet.

 Item-based Collaborative Filtering:

 * This approach recommends items that are similar to the ones the user has already liked or rated highly.

 * It assumes: "If Anime X and Anime Y are similar, and you liked Anime X, you’ll probably like Anime Y."

 * Example: If you liked "Naruto", and many users who liked Naruto also liked "Bleach", you may get a recommendation for "Bleach".

2. What is collaborative filtering, and how does it work?
Collaborative Filtering is a technique used in recommendation systems to suggest items (like movies, products, or anime) based on the preferences and behaviors of other users.

How It Works:

* It uses user-item interaction data (e.g., ratings, clicks, watch history).

* The core idea is that users who agreed in the past will likely agree again in the future.

* There are two main types:

  * User-based: Finds users with similar preferences.

  * Item-based: Finds items that are liked together by users.

* Similarity is typically calculated using metrics like:

  * Cosine similarity

  * Pearson correlation

  * Jaccard index